In [1]:
import pandas as pd
import numpy as np

df_resale = pd.read_csv("/Users/mulan/NUS/AY24_Y3S2/DSE3101/DSE3101-HDB-Decode/data/cleaned/ResalePrice_amenities_coor.csv")

## Adjust resale price based on inflation rate

## Convert lease_commence_date into building age by 2025

In [2]:
df_resale['building_age_2025'] = 2025 - df_resale['lease_commence_date']

## Convert to numeric 

In [3]:
# Convert remaining_lease into numeric variables
def convert_to_years(age_str):
    parts = age_str.split()
    years = int(parts[0])
    months = 0
    if len(parts) > 2:
        months = int(parts[2])
    return round(years + (months / 12),2)

df_resale['remaining_lease'] = df_resale['remaining_lease'].apply(convert_to_years)

In [4]:
flat_mapping = {
    '1 ROOM': 1,
    '2 ROOM': 2,
    '3 ROOM': 3,
    '4 ROOM': 4,
    '5 ROOM': 5,
    'EXECUTIVE': 6,
    'MULTI-GENERATION': 7
}

df_resale['flat_type'] = df_resale['flat_type'].map(flat_mapping)


In [5]:
df_resale.drop(columns=['address','street_name', 'block', 'LatLng', 'lease_commence_date'])
df_resale["year"] = df_resale["month"].str[:4].astype(int)

## Add Macro variables

In [6]:
df_unemployment = pd.read_csv("/Users/mulan/NUS/AY24_Y3S2/DSE3101/DSE3101-HDB-Decode/data/raw/unemployment.csv")
df_interest = pd.read_csv("/Users/mulan/NUS/AY24_Y3S2/DSE3101/DSE3101-HDB-Decode/data/raw/interest_rate.csv",skiprows=10)
df_fxrate = pd.read_csv("/Users/mulan/NUS/AY24_Y3S2/DSE3101/DSE3101-HDB-Decode/data/raw/fxrate.csv")

### Unemployment Rate

In [7]:
# Choose relevant rows
df_unemployment = df_unemployment.iloc[9:12]
df_unemployment.columns = df_unemployment.iloc[0]

# Pivot the columns
df_unemployment = df_unemployment[1:]
df_unemployment = df_unemployment.melt(id_vars=["Data Series"], var_name="year", value_name="unemployment_rate")
df_unemployment["year"] = df_unemployment["year"].astype(int)
df_unemployment = df_unemployment.pivot(index="year", columns="Data Series", values="unemployment_rate").reset_index()
df_unemployment.columns = ["year", "total_unemployment_rate", "resident_unemployment_rate"]

# Join with resale dataframe
df_resale = df_resale.merge(df_unemployment, on="year", how="left")
df_resale


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,LatLng,education_score,shopping_score,food_score,recreation_score,healthcare_score,building_age_2025,year,total_unemployment_rate,resident_unemployment_rate
0,2017-03,BUKIT MERAH,3,43,TELOK BLANGAH RISE,13 TO 15,59.0,Improved,1976,58.08,...,"1.270379512,103.823236",9.886121,31.231448,14.302518,29.626696,55.974066,49,2017,3.1,2.2
1,2017-05,BUKIT MERAH,3,43,TELOK BLANGAH RISE,07 TO 09,59.0,Improved,1976,57.92,...,"1.270379512,103.823236",9.886121,31.231448,14.302518,29.626696,55.974066,49,2017,3.1,2.2
2,2018-01,BUKIT MERAH,3,43,TELOK BLANGAH RISE,10 TO 12,59.0,Improved,1976,57.25,...,"1.270379512,103.823236",9.886121,31.231448,14.302518,29.626696,55.974066,49,2018,2.9,2.1
3,2018-06,BUKIT MERAH,3,43,TELOK BLANGAH RISE,01 TO 03,59.0,Improved,1976,56.92,...,"1.270379512,103.823236",9.886121,31.231448,14.302518,29.626696,55.974066,49,2018,2.9,2.1
4,2018-06,BUKIT MERAH,3,43,TELOK BLANGAH RISE,04 TO 06,59.0,Improved,1976,56.92,...,"1.270379512,103.823236",9.886121,31.231448,14.302518,29.626696,55.974066,49,2018,2.9,2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201055,2024-07,SEMBAWANG,5,484,ADMIRALTY LINK,07 TO 09,110.0,Improved,2003,78.08,...,"1.457071216,103.8153084",19.199755,30.596328,1.233127,25.048367,8.851718,22,2024,2.7,2.0
201056,2024-08,SEMBAWANG,4,484,ADMIRALTY LINK,13 TO 15,91.0,Model A,2003,78.00,...,"1.457071216,103.8153084",19.199755,30.596328,1.233127,25.048367,8.851718,22,2024,2.7,2.0
201057,2024-10,SEMBAWANG,5,484,ADMIRALTY LINK,13 TO 15,111.0,Improved,2003,77.83,...,"1.457071216,103.8153084",19.199755,30.596328,1.233127,25.048367,8.851718,22,2024,2.7,2.0
201058,2024-10,SEMBAWANG,5,484,ADMIRALTY LINK,07 TO 09,110.0,Improved,2003,77.83,...,"1.457071216,103.8153084",19.199755,30.596328,1.233127,25.048367,8.851718,22,2024,2.7,2.0


### Interest rate

Choose Singapore Overnight Rate Average (SORA) as the indicator for interest rate/

In [8]:
df_interest = df_interest[:8]
df_interest = df_interest.iloc[[6]]
df_interest

,Data Series,2025 Jan,2024 Dec,2024 Nov,2024 Oct,2024 Sep,2024 Aug,2024 Jul,2024 Jun,2024 May,...,1988 Oct,1988 Sep,1988 Aug,1988 Jul,1988 Jun,1988 May,1988 Apr,1988 Mar,1988 Feb,1988 Jan
6,Singapore Overnight Rate Average,2.6325,2.1123,3.0839,3.1559,3.8808,3.5355,3.4336,3.4325,3.5222,...,na,na,na,na,na,na,na,na,na,na


In [9]:
# Remove NA
df_interest.replace('na', np.nan, inplace=True)
df_interest = df_interest.dropna(axis=1)


df_interest = df_interest.melt(var_name="month", value_name="interest_rate")
df_interest = df_interest[1:]

/var/folders/pf/xn094ngs70lb3jh0xpvlzx8c0000gn/T/ipykernel_57097/888357619.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_interest.replace('na', np.nan, inplace=True)


### Foreign Exchange Rate

In [10]:
df_interest["month"] = pd.to_datetime(df_interest["month"], format="%Y %b").dt.strftime("%Y-%m")

ValueError: unconverted data remains when parsing with format "%Y %b": " ", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
df_interest

,month,interest_rate
1,2025 Jan,2.6325
2,2024 Dec,2.1123
3,2024 Nov,3.0839
4,2024 Oct,3.1559
5,2024 Sep,3.8808
...,...,...
231,2005 Nov,2.8966
232,2005 Oct,2.5551
233,2005 Sep,2.2293
234,2005 Aug,1.9718


In [ ]:
fxrate = pd.read_csv("../data/raw/fxrate.csv", skiprows=9).head(1)
fxrate

,Data Series,2025 Jan,2024 Dec,2024 Nov,2024 Oct,2024 Sep,2024 Aug,2024 Jul,2024 Jun,2024 May,...,1988 Oct,1988 Sep,1988 Aug,1988 Jul,1988 Jun,1988 May,1988 Apr,1988 Mar,1988 Feb,1988 Jan
0,US Dollar (Singapore Dollar Per US Dollar),1.3625,1.3503,1.3365,1.309,1.2964,1.3159,1.3469,1.3516,1.3512,...,2.0214,2.0398,2.0411,2.0462,2.0269,2.0098,2.0037,2.0133,2.0173,2.0253


## Normalise continuous variables